## Convolutional NN to classify govuk content to level2 taxons

Based on:
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

### Load requirements and data

In [48]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, layer_utils, plot_model
from keras.layers import (Embedding, Input, Dense, 
                          Activation, Conv1D, MaxPooling1D, Flatten)
from keras.models import Model, Sequential
from keras.optimizers import rmsprop
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


In [3]:
labelled_level2 = pd.read_csv('../../../data/labelled_level2.csv', dtype=object)

## Hyperparameters

In [4]:
#MAX_NB_WORDS
MAX_SEQUENCE_LENGTH =1000
#EMBEDDING_DIM

### Create target/Y 

Note: when using the categorical_crossentropy loss, your targets should be in categorical format (e.g. if you have 10 classes, the target for each sample should be a 10-dimensional vector that is all-zeros expect for a 1 at the index corresponding to the class of the sample). In order to convert integer targets into categorical targets, you can use the Keras utility to_categorical

In [5]:
#COLLAPSE World level2taxons
labelled_level2.loc[labelled_level2['level1taxon'] == 'World', 'level2taxon'] = 'world_level1'

In [6]:
labelled_level2['level2taxon'] = labelled_level2['level2taxon'].astype('category')

labels = labelled_level2['level2taxon'].cat.codes 


In multilabel learning, the joint set of binary classification tasks is expressed with label binary indicator array: each sample is one row of a 2d array of shape (n_samples, n_classes) with binary values:  
the one, i.e. the non zero elements, corresponds to the subset of labels.  
An array such as np.array([[1, 0, 0], [0, 1, 1], [0, 0, 0]]) represents label 0 in the first sample, labels 1 and 2 in the second sample, and no labels in the third sample.  
Producing multilabel data as a list of sets of labels may be more intuitive. The MultiLabelBinarizer transformer can be used to convert between a collection of collections of labels and the indicator format.

####  first reshape wide to get columns for each level2taxon and row number = number unique urls

In [7]:
#get a smaller copy of data for pivoting ease (think you can work from full data actually and other cols get droopedauto)

level2_reduced = labelled_level2[['content_id', 'level2taxon', 'combined_text']].copy()

#how many level2taxons are there?
print(level2_reduced.level2taxon.nunique())

#count the number of taxons per content item into new column
level2_reduced['num_taxon_per_content'] = level2_reduced.groupby(["content_id"])['content_id'].transform("count")

#Add 1 because of zero-indexing to get 1-number of level2taxons as numerical targets
level2_reduced['level2taxon_code'] = level2_reduced.level2taxon.astype('category').cat.codes + 1

210


In [8]:
#reshape to wide per taxon and keep the combined text so indexing is consistent when splitting X from Y

multilabel = (level2_reduced.pivot_table(index=['content_id', 'combined_text'], 
                  columns='level2taxon_code', 
                  values='num_taxon_per_content'))
print(level2_reduced.shape)
print(multilabel.shape)

list(multilabel.columns)

(173560, 5)
(114048, 210)


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [9]:
#convert the number_of_taxons_per_content values to 1, meaning there was an entry for this taxon and this content_id, 0 otherwise
binary_multilabel = multilabel.notnull().astype('int')

In [10]:
#will convert columns to an array of shape
binary_multilabel[list(binary_multilabel.columns)].values.shape

(114048, 210)

In [11]:
#convert columns to an array. Each row represents a content item, each column an individual taxon
binary_multilabel = binary_multilabel[list(binary_multilabel.columns)].values
binary_multilabel[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [12]:
type(binary_multilabel)

numpy.ndarray

In [13]:
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(binary_multilabel)
# y.shape

In [14]:
#Use this for singlelabel problems
labels = to_categorical(np.asarray(labels))

print('Shape of label tensor:', labels.shape)

labels

Shape of label tensor: (173560, 210)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

### Create language data/X

format our text samples and labels into tensors that can be fed into a neural network. To do this, we will rely on Keras utilities keras.preprocessing.text.Tokenizer and keras.preprocessing.sequence.pad_sequences.

In [15]:
multilabel.index.names
texts = multilabel.index.get_level_values('combined_text')
texts.shape

(114048,)

In [16]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts) #yield one sequence per input text

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen= MAX_SEQUENCE_LENGTH) #MAX_SEQUENCE_LENGTH

Found 213132 unique tokens.


In [17]:
print('Shape of label tensor:', binary_multilabel.shape)
print('Shape of data tensor:', data.shape)

Shape of label tensor: (114048, 210)
Shape of data tensor: (114048, 1000)


### Data split

In [18]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = binary_multilabel[indices]
nb_validation_samples = int(0.2 * data.shape[0]) #validation split
print('nb_validationsamples:', nb_validation_samples)


print ("number of training examples = " + str(x_train.shape[0]))
x_train = data[:-nb_validation_samples]
print('Shape of x_train:', x_train.shape)
y_train = labels[:-nb_validation_samples]
print('Shape of y_train:', y_train.shape)
x_val = data[-nb_validation_samples:]
print('Shape of x_val:', x_val.shape)
y_val = labels[-nb_validation_samples:]
print('Shape of y_val:', y_val.shape)

nb_validationsamples: 22809
Shape of x_train: (91239, 1000)
Shape of y_train: (91239, 210)
Shape of x_val: (22809, 1000)
Shape of y_val: (22809, 210)


### preparing the Embedding layer
compute an index mapping words ot known embeddings by parsing the data dump of pre-trained embeddings
NB stopwords haven't been removed yet...

In [19]:
embeddings_index = {}
f = open(os.path.join('../../../data/', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


compute embedding matrix using embedding_index dict and word_index

In [20]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))# used 6B.100d.txt
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.

In [21]:
embedding_layer = Embedding(len(word_index) + 1,
                            100, # used 6B.100d.txt
                            weights=[embedding_matrix],
                            input_length= MAX_SEQUENCE_LENGTH, #MAX_SEQUENCE LENGTH
                            trainable=False)

An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). These input sequences should be padded so that they all have the same length in a batch of input data (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't pass an explicit input_length argument to the layer).

All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. This means that the output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).

## Training a 1D convnet

In [22]:
NB_CLASSES = y_train.shape[1]
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') #MAX_SEQUENCE_LENGTH
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(128, 5, activation='relu', name = 'conv0')(embedded_sequences)

x = MaxPooling1D(5, name = 'max_pool0')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv1')(x)

x = MaxPooling1D(5 , name = 'max_pool1')(x)

x = Conv1D(128, 5, activation='relu', name = 'conv2')(x)

x = MaxPooling1D(35, name = 'global_max_pool')(x)  # global max pooling

x = Flatten()(x) #reduce dimensions from 3 to 2; convert to vector + FULLYCONNECTED

x = Dense(128, activation='relu')(x)

x = Dense(NB_CLASSES, activation='sigmoid', name = 'fully_connected')(x)

# Create model. 
# This creates Keras model instance, will use this instance to train/test the model.
model = Model(sequence_input, x)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Metric values are recorded at the end of each epoch on the training dataset. If a validation dataset is also provided, then the metric recorded is also calculated for the validation dataset.

All metrics are reported in verbose output and in the history object returned from calling the fit() function. In both cases, the name of the metric function is used as the key for the metric values. In the case of metrics for the validation dataset, the “val_” prefix is added to the key.

You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
1. Create the model by calling the function above
2. Compile the model by calling `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Train the model on train data by calling `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`
4. Test the model on test data by calling `model.evaluate(x = ..., y = ...)`

If you want to know more about `model.compile()`, `model.fit()`, `model.evaluate()` and their arguments, refer to the official [Keras documentation](https://keras.io/models/model/).


In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         21313300  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048     
__________

In [24]:
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)

In [57]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
         self.val_f1s = []
         self.val_recalls = []
         self.val_precisions = []
 
    def on_epoch_end(self, epoch, logs={}):
         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
         val_targ = self.model.validation_data[1]
         _val_f1 = f1_score(val_targ, val_predict)
         _val_recall = recall_score(val_targ, val_predict)
         _val_precision = precision_score(val_targ, val_predict)
         self.val_f1s.append(_val_f1)
         self.val_recalls.append(_val_recall)
         self.val_precisions.append(_val_precision)
         print("- val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
    return
 
metrics = Metrics()

SyntaxError: 'return' outside function (<ipython-input-57-1fce7e76d9f2>, line 17)

In [25]:
x_train.shape

(91239, 1000)

In [26]:
y_train.shape

(91239, 210)

In [27]:
model.fit(
    x_train, y_train, 
    validation_data=(x_val, y_val), 
    epochs=4, batch_size=128, 
    callbacks=[metrics]
)

Train on 91239 samples, validate on 22809 samples
Epoch 1/4
91239/91239 [==============================] - 2248s 25ms/step - loss: 0.0394 - acc: 0.9913 - val_loss: 0.0248 - val_acc: 0.9934
Epoch 2/4
91239/91239 [==============================] - 2186s 24ms/step - loss: 0.0205 - acc: 0.9941 - val_loss: 0.0183 - val_acc: 0.9944
Epoch 3/4
91239/91239 [==============================] - 2195s 24ms/step - loss: 0.0161 - acc: 0.9950 - val_loss: 0.0157 - val_acc: 0.9951
Epoch 4/4
91239/91239 [==============================] - 2185s 24ms/step - loss: 0.0141 - acc: 0.9955 - val_loss: 0.0139 - val_acc: 0.9956


In [51]:
preds = model.predict(x_val)

preds[preds>=0.5] = 1
preds[preds<0.5] = 0

f1_score(y_val, preds, average='macro')

/Users/ellieking/Documents/tag_tax/govuk-taxonomy-supervised-learning/tax_SL/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ellieking/Documents/tag_tax/govuk-taxonomy-supervised-learning/tax_SL/lib/python3.4/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.15872847003360904

In [33]:
import pydot

In [35]:
from IPython.display import SVG

In [36]:
from keras.utils.vis_utils import model_to_dot

In [37]:
from keras.utils import plot_model

In [40]:
!pip3 install kt_utils
from kt_utils import *

  Could not find a version that satisfies the requirement kt_utils (from versions: )
No matching distribution found for kt_utils


ImportError: No module named 'kt_utils'

In [41]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [43]:
plot_model(model, to_file='CNN_model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [44]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

Improve accuracy by training longer with some regularization mechanism (such as dropout) or by fine-tuning the Embedding layer.

We can also test how well we would have performed by not using pre-trained word embeddings, but instead initializing our Embedding layer from scratch and learning its weights during training. We just need to replace our Embedding layer with the following:

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH)
                            
After 2 epochs, this approach only gets us to 90% validation accuracy, less than what the previous model could reach in just one epoch. Our pre-trained embeddings were definitely buying us something. In general, using pre-trained embeddings is relevant for natural processing tasks were little training data is available (functionally the embeddings act as an injection of outside information which might prove useful for your model).



In [28]:
# Build a classifier optimized for maximizing f1_score (uses class_weights)
# from keras.layers import Dropout

# clf = Sequential()

# clf.add(Dense(x_train.shape[1], 1600, activation='relu'))
# clf.add(Dropout(0.6))
# clf.add(Dense(1600, 1200, activation='relu'))
# clf.add(Dropout(0.6))
# clf.add(Dense(1200, 800, activation='relu'))
# clf.add(Dropout(0.6))
# clf.add(Dense(800, y_train.shape[1], activation='sigmoid'))

# clf.compile(optimizer=Adam(), loss='binary_crossentropy')

# clf.fit(xt, yt, batch_size=64, nb_epoch=300, validation_data=(xs, ys), class_weight=W, verbose=0)

# preds = clf.predict(xs)

# preds[preds>=0.5] = 1
# preds[preds<0.5] = 0

# print(f1_score(ys, preds, average='macro'))